# Resolve Schema

**input**
```
+schema_path: str
```

**methods**
```
+read_json(schema_path: str): dict
+split_json(schema: dict): list
+resolver(entity1: dict, entity2: dict): dict
+resolve_defs(terms: dict, defs: dict) : dict
+ node_order(schema: dict): list
+resolve_nodes(nodeList: list, splitJsonList: list): list
+recombine_nodes(resolvedList: list) : dict
```

In [1]:
import gen3_data_validator

In [2]:
ResolverInst = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
# DataInst = gen3_data_validator.ParseData(data_folder_path = "../data/fail")
# LinkageInst = gen3_data_validator.TestLinkage(schema_resolver = ResolverInst, data_parser = DataInst)
node_lookup = ResolverInst.generate_node_lookup()

DataInst = gen3_data_validator.ParseXlsxMetadata(xlsx_path= "../data/restricted/edcad_test.xlsx")

=== Resolving Schema References ===
Resolved demographic.yaml
Resolved project.yaml
Resolved serum_marker_assay.yaml
Resolved alignment_workflow.yaml
Resolved imaging_file.yaml
Resolved lipidomics_assay.yaml
Resolved metabolomics_file.yaml
Resolved acknowledgement.yaml
Resolved medical_history.yaml
Resolved _settings.yaml
Resolved blood_pressure_test.yaml
Resolved genomics_assay.yaml
Resolved variant_file.yaml
Resolved program.yaml
Resolved serum_marker_file.yaml
Resolved proteomics_assay.yaml
Resolved sample.yaml
Resolved unaligned_reads_file.yaml
Resolved aligned_reads_index_file.yaml
Resolved variant_workflow.yaml
Resolved proteomics_file.yaml
Resolved exposure.yaml
Resolved metabolomics_assay.yaml
Resolved lipidomics_mapping_file.yaml
Resolved lipidomics_file.yaml
Resolved aligned_reads_file.yaml
Resolved lab_result.yaml
Resolved medication.yaml
Resolved publication.yaml
Resolved subject.yaml
Resolved core_metadata_collection.yaml


AttributeError: 'ParseXlsxMetadata' object has no attribute 'skip_rows'

In [4]:
LinkageInst.generate_config(DataInst.data_dict)

{'metabolomics_file': {'primary_key': 'metabolomics_files',
  'foreign_key': 'metabolomics_assays'},
 'medical_history': {'primary_key': 'medical_historys',
  'foreign_key': 'subjects'},
 'metabolomics_assay': {'primary_key': 'metabolomics_assays',
  'foreign_key': 'samples'},
 'sample': {'primary_key': 'samples', 'foreign_key': 'subjects'},
 'subject': {'primary_key': 'subjects', 'foreign_key': None}}

In [5]:
# Using the Linkage Inst which has the resolved schema with custom data and config
config_map = {
    "samples": {"primary_key": "sample_id", "foreign_key": "subject_id"},
    "files": {"primary_key": "file_id", "foreign_key": "sample_id"},
    "subjects": {"primary_key": "subject_id", "foreign_key": "project_id"},
    "project": {"primary_key": "project_id", "foreign_key": None}
}

data_map = {
    "samples": [
        {"sample_id": "sample_1", "subject_id": "subject_9"},
        {"sample_id": "sample_2", "subject_id": "subject_3"},  # Invalid FK
        {"sample_id": "sample_3", "subject_id": "subject_4"}, # Invalid FK
        {"sample_id": "sample_4", "subject_id": "subject_5"} # Invalid FK
    ],
    "files": [
        {"file_id": "file_1", "sample_id": "sample_1"},
        {"file_id": "file_2", "sample_id": "sample_27"}  # Invalid FK
    ],
    "subjects": [
        {"subject_id": "subject_1", "project_id": "project_1"},  
        {"subject_id": "subject_2", "project_id": "project_2"}, # Missing project 2
    ],
    "project": [
        {"project_id": "project_1"}
    ]
}

LinkageInst.validate_links(data_map, config_map)

=== Validating Config Map ===
Root Node = ['subject']
Config Map Validated
=== Validating Links ===
Entity 'samples' has 4 invalid foreign keys: ['subject_9', 'subject_3', 'subject_4', 'subject_5']
Entity 'files' has 1 invalid foreign keys: ['sample_27']
Entity 'subjects' has 1 invalid foreign keys: ['project_2']
Entity 'project' has 0 invalid foreign keys: []


{'samples': ['subject_9', 'subject_3', 'subject_4', 'subject_5'],
 'files': ['sample_27'],
 'subjects': ['project_2'],
 'project': []}

In [6]:
ResolverInst = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
xlsxData = gen3_data_validator.ParseXlsxMetadata(xlsx_path = "../data/restricted/ausdiab_lipid_manifest.xlsx")


=== Resolving Schema References ===
Resolved demographic.yaml
Resolved project.yaml
Resolved serum_marker_assay.yaml
Resolved alignment_workflow.yaml
Resolved imaging_file.yaml
Resolved lipidomics_assay.yaml
Resolved metabolomics_file.yaml
Resolved acknowledgement.yaml
Resolved medical_history.yaml
Resolved _settings.yaml
Resolved blood_pressure_test.yaml
Resolved genomics_assay.yaml
Resolved variant_file.yaml
Resolved program.yaml
Resolved serum_marker_file.yaml
Resolved proteomics_assay.yaml
Resolved sample.yaml
Resolved unaligned_reads_file.yaml
Resolved aligned_reads_index_file.yaml
Resolved variant_workflow.yaml
Resolved proteomics_file.yaml
Resolved exposure.yaml
Resolved metabolomics_assay.yaml
Resolved lipidomics_mapping_file.yaml
Resolved lipidomics_file.yaml
Resolved aligned_reads_file.yaml
Resolved lab_result.yaml
Resolved medication.yaml
Resolved publication.yaml
Resolved subject.yaml
Resolved core_metadata_collection.yaml


In [7]:
xlsxData.write_dict_to_json(output_dir="../data/restricted/ausdiab_lipid_metadata/")

JSON files written to ../data/restricted/ausdiab_lipid_metadata/


In [8]:
DataInst = gen3_data_validator.ParseData(data_folder_path = "../data/restricted/ausdiab_lipid_metadata/")
LinkageInst = gen3_data_validator.TestLinkage(schema_resolver = ResolverInst, data_parser = DataInst)

=== Validating Config Map ===
Root Node = ['subject']
Config Map Validated
=== Validating Links ===
Entity 'lipidomics_assay' has 0 invalid foreign keys: []
Entity 'lipidomics_mapping_file' has 0 invalid foreign keys: []
Entity 'sample' has 0 invalid foreign keys: []
Entity 'subject' has 10339 invalid foreign keys: ['project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdiab-001', 'project-ausdia

# validation prototype


In [10]:
from jsonschema import Draft4Validator
from functools import wraps
from datetime import datetime
from functools import wraps
from time import time


class Validate:
    
    def __init__(self, data_map, resolved_schema):
        self.data_map = data_map
        self.resolved_schema = resolved_schema
        self.validation_result = self.validate_schema(self.data_map, self.resolved_schema)
    
    
    def validate_object(self, obj, idx, validator) -> list:
        """
        Validates a single JSON object against a provided JSON schema validator.

        Parameters:
        - obj (dict): The JSON object to validate.
        - idx (int): The index of the object in the dataset.
        - validator (Draft4Validator): The JSON schema validator to use for validation.

        Returns:
        - list: A list of dictionaries containing validation results and log messages.
        """
        validation_results = []
        errors = list(validator.iter_errors(obj))

        if len(errors[1:]) == 0:
            result = {
                "index": idx,
                "validation_result": "PASS",
                "invalid_key": None,
                "schema_path": None,
                "validator": None,
                "validator_value": None,
                "validation_error": None
            }
            validation_results.append(result)
            # print(f"=== SUCCESS === | object {idx} | No errors found")
        else:
            for error in errors[1:]:
                invalid_key = ".".join(str(k) for k in error.path) if error.path else "root"
                schema_path = ".".join(str(k) for k in error.schema_path)

                result = {
                    "index": idx,
                    "validation_result": "FAIL",
                    "invalid_key": invalid_key,
                    "schema_path": schema_path,
                    "validator": error.validator,
                    "validator_value": error.validator_value,
                    "validation_error": error.message
                }
                validation_results.append(result)
                # print(f"=== FAIL === | object {idx} | Error in {invalid_key}")

        return validation_results
    
    
    def _analyze_validation_results(self, validation_result_input: dict, node: str) -> dict:
        
        def pass_in_dict(input: dict) -> bool:
            return input["validation_result"] == "PASS"

        def pass_in_list_dict(input: list) -> list:
            return [pass_in_dict(obj) for obj in input]

        number_of_objects = len(validation_result_input[node])
        number_of_pass = sum(
            1 for index_block in validation_result_input[node] if True in pass_in_list_dict(index_block)
        )

        return {
            "number_of_objects": number_of_objects,
            "number_of_pass": number_of_pass
        }
        

    def validate_schema(self, data_map: dict, resolved_schema: dict) -> dict:
        """
        Takes in a dictionary of data, where the key is the entity name, and the value is a list of jsons containing the data.
        The function then validates the data against the resolved schema.
        
        Args:
        - data_map (dict): A dictionary where keys are entity names and values are lists of JSON objects to be validated.
        - resolved_schema (dict): A dict of resolved JSON schema objects to validate against.

        Returns:
        - dict: A dictionary containing validation results for each entity.
        """
        # Initialize counters and error storage
        validation_results = {}
        
        data_nodes = list(data_map.keys())
        print(f"Data nodes: {data_nodes}")
        schema_keys = [key[:-5] if key.endswith('.yaml') else key for key in resolved_schema.keys()]
        print(f"Schema keys: {schema_keys}")
        
        for node in data_nodes:
            if node not in schema_keys:
                print(f"Warning: {node} not found in resolved schema keys.")
                continue
        
            data = data_map[node]
            schema = resolved_schema[f"{node}.yaml"]
            validator = Draft4Validator(schema)

            node_results = []
            for idx, obj in enumerate(data):
                result = self.validate_object(obj, idx, validator)
                node_results.append(result)
                validation_results[node] = node_results
                
        
        for node in validation_results.keys(): 
            validation_totals = self._analyze_validation_results(validation_results, node)
            print(f"=== Validation Summary for {node} ===")
            print(f"Total: {validation_totals['number_of_objects']}")
            print(f"Success: {validation_totals['number_of_pass']}")
            print(f"Fail: {validation_totals['number_of_objects'] - validation_totals['number_of_pass']}")
        
        
        
        return validation_results

            
# next step is to add summary stats so we can identify unique validation errors

In [12]:
import gen3_data_validator
ResolverInst = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
DataInst = gen3_data_validator.ParseData(data_folder_path = "../data/restricted/ausdiab_lipid_metadata")
ValidationInst = Validate(data_map=DataInst.data_dict, resolved_schema=ResolverInst.schema_resolved)
ValidationInst.validation_result['sample']

=== Resolving Schema References ===
Resolved demographic.yaml
Resolved project.yaml
Resolved serum_marker_assay.yaml
Resolved alignment_workflow.yaml
Resolved imaging_file.yaml
Resolved lipidomics_assay.yaml
Resolved metabolomics_file.yaml
Resolved acknowledgement.yaml
Resolved medical_history.yaml
Resolved _settings.yaml
Resolved blood_pressure_test.yaml
Resolved genomics_assay.yaml
Resolved variant_file.yaml
Resolved program.yaml
Resolved serum_marker_file.yaml
Resolved proteomics_assay.yaml
Resolved sample.yaml
Resolved unaligned_reads_file.yaml
Resolved aligned_reads_index_file.yaml
Resolved variant_workflow.yaml
Resolved proteomics_file.yaml
Resolved exposure.yaml
Resolved metabolomics_assay.yaml
Resolved lipidomics_mapping_file.yaml
Resolved lipidomics_file.yaml
Resolved aligned_reads_file.yaml
Resolved lab_result.yaml
Resolved medication.yaml
Resolved publication.yaml
Resolved subject.yaml
Resolved core_metadata_collection.yaml
Data nodes: ['lipidomics_assay', 'lipidomics_mappin

[[{'index': 0,
   'validation_result': 'FAIL',
   'invalid_key': 'alternate_timepoint',
   'schema_path': 'properties.alternate_timepoint.type',
   'validator': 'type',
   'validator_value': 'string',
   'validation_error': "None is not of type 'string'"},
  {'index': 0,
   'validation_result': 'FAIL',
   'invalid_key': 'freeze_thaw_cycles',
   'schema_path': 'properties.freeze_thaw_cycles.type',
   'validator': 'type',
   'validator_value': 'integer',
   'validation_error': "'The samples were freezer thawed not all the same number of times, due to the different volumes that were in the tubes.' is not of type 'integer'"},
  {'index': 0,
   'validation_result': 'FAIL',
   'invalid_key': 'sample_id',
   'schema_path': 'properties.sample_id.type',
   'validator': 'type',
   'validator_value': 'string',
   'validation_error': "101 is not of type 'string'"},
  {'index': 0,
   'validation_result': 'FAIL',
   'invalid_key': 'sample_in_preservation',
   'schema_path': 'properties.sample_in_pre

In [ ]:
ValidationInst.validation_result

In [34]:
# Use this for writing tests

sample_validation_results = {
    'sample': [
        [
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'10' is not of type 'integer'"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_provider',
                'schema_path': 'properties.sample_provider.enum',
                'validator': 'enum',
                'validator_value': ['Baker', 'USYD', 'UMELB', 'UQ'],
                'validation_error': "45 is not one of ['Baker', 'USYD', 'UMELB', 'UQ']"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'Autoclave' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'76' is not of type 'integer'"
            },
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'In the Pantry' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 2,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ],
        [
            {
                'index': 3,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ]
    ]
}




Number of objects: 4
Number of passes: 2


# Testing if a xlsx with no links can parse

In [16]:
# edcadXlsxData = gen3_data_validator.ParseXlsxMetadata(xlsx_path = "../data/restricted/edcad_test.xlsx")
edcadXlsxData.write_dict_to_json(output_dir = "../data/restricted/edcad_test")

FileNotFoundError: [Errno 2] No such file or directory: "../data/restricted/edcad_test/ACDCHarm'n_EDCAD merged.json"

In [17]:
edcadXlsxData.xlsx_data_dict

{"ACDCHarm'n_EDCAD merged":    Record ID site baseline_date  month_birth  year_birth  baseline_age  \
 1          3  MEL    2020-12-07           11        1962            58   
 2          4  MEL    2020-12-12            2        1966            54   
 3          5  MEL    2021-01-06            3        1954            66   
 4          6  MEL    2020-12-11            6        1960            60   
 5          7  MEL    2020-12-07            4        1970            50   
 6          9  MEL    2020-11-27            8        1968            52   
 
       sex  bmi_baseline hypertension_self_reported  bp_systolic  ...  \
 1    male          23.8                         no        128.0  ...   
 2  female           NaN                         no        128.0  ...   
 3  female          36.3                        yes        147.0  ...   
 4    male          24.7                         no        135.0  ...   
 5    male          24.4                         no        122.0  ...   
 6  fema

@startuml
class ResolveSchema {
    read_json()
    get_nodes()
    get_node_link()
    get_node_category()
    get_node_properties()
    generate_node_lookup()
    find_upstream_downstream()
    get_all_node_pairs()
    get_node_order()
    split_json()
    return_schema()
    resolve_references()
    resolve_node()
    schema_list_to_json()
    resolve_all_references()
    return_resolved_schema()
        schema_path
    schema
    nodes
    node_pairs
    node_order
    schema_list
    schema_def
    schema_term
    schema_def_resolved
    schema_list_resolved
    schema_resolved
    __init__()
}
@enduml